This Jupyter Notebook serves as a proof of concept of a multi-agent solution for MedTech regulations. The intent of the system is to provide clear answers to questions on WHO and FDA documentations on medical devices, using a 3-to-4 agents system composed like this:

- an LLM orchestrator that receives the question and coordinates agents
- a RAG agent capable of retrieving documents related to the question
- an LLM response agent to put together the answer based on the documents retrieved and the question
- a possible fourth agent to be decided (summary agent, source-verifier, compare agent, prompt agent to improve prompts, etc.)

In [1]:
"""

pseudo code for multi agent system

class agent

instantiate orchestrator and other agents

define orchestrator prompt and response false

take input

while not response:

    orchestrator call

    designed agent call

return response

"""

'\n\npseudo code for multi agent system\n\nclass agent\n\ninstantiate orchestrator and other agents\n\ndefine orchestrator prompt and response false\n\ntake input\n\nwhile not response:\n\n    orchestrator call\n\n    designed agent call\n\nreturn response\n\n'

In [53]:
# Import libraries

import gradio as gr
import chromadb
import uuid
from pypdf import PdfReader
import google.generativeai as genai
import os
import time
from dotenv import load_dotenv
import re
import pymupdf4llm
import json

In [9]:
# Load environmental variables and AI models

load_dotenv() #loads the API key put in a .env file
try:
    genai.configure(api_key=os.environ['GOOGLE_API_KEY'])
except Exception as e:
    print(f"Error configuring Google AI. Please ensure your API key is correct. Error: {e}")

# Configure paths to data
current_dir = os.getcwd() 
FILE_PATH_FDA_DESIGN = os.path.join(current_dir, '..', 'documents', 'FDA_Design_Control_Guidance.pdf')
FILE_PATH_WHO = os.path.join(current_dir, '..', 'documents', 'WHO_Medical_Device_Regulations.pdf')
FILE_PATH_FDA_POLICY = os.path.join(current_dir, '..', 'documents', 'FDA_Policy_Device_Software_Functions.pdf')
COLLECTION_NAME = "multi_agent_rag"

In [29]:
# Split logic (very simple, splitting paragraphs)

def split_into_chunks(dict_list):
    text_chunks = []
    metadatas = []
    
    for document in dict_list:
        try:
            paragraphs = list(document.values())[0].split("\n\n")
    
            for paragraph in paragraphs: # Delete short paragraphs
                if len(paragraph) > 10:
                    text_chunks.append(paragraph)
                    metadatas.append({"source": list(document.keys())[0]})
        except Exception as e:
            print(f"An error occurred while processing {list(document.keys())[0]}: {e}")
            
    return text_chunks, metadatas

# Use pymupdf4llm to convert pdf into text, adequately formatted. Using dicts to keep track of sources and add them to metadata while chunking
fda_design_dict = {'FDA_Design_Control_Guidance.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_DESIGN)}
who_dict = {'WHO_Medical_Device_Regulations.pdf': pymupdf4llm.to_markdown(FILE_PATH_WHO)}
fda_policy_dict = {'FDA_Policy_Device_Software_Functions.pdf': pymupdf4llm.to_markdown(FILE_PATH_FDA_POLICY)}

text_chunks, metadatas = split_into_chunks([fda_design_dict, who_dict, fda_policy_dict])
print(len(text_chunks), len(metadatas))

1657 1657


In [83]:
# Define agent classes

class OrchestratorAgent:

    def __init__(self):
        self.prompt = """
        Act as an orchestrator agent for an intelligent RAG system for MedTech companies. Your task is to coordinate agents in order to extract relevant documents from a RAG system and package a coherent and precise answer to the query received.
        The task is to call a first time the ragagent, and then use the ragagent documents to call the response_agent on them and craft a response. Make sure to call the response_agent if the history contains a previous call to the ragagent.
        Your AI agents are: 
        - ragagent: the agent responsible for querying the RAG system and returning relevant documents and scores, based on the query. Expects a query as input and outputs documents, metadatas and sources. 
        - response_agent: the agent who will craft the response based on the query and the relevant documents provided by the ragagent. Expects a query and documents as inputs and outputs a string.

        You will return instructions in a JSON format. For the ragagent will be like:
        {
            "agent_name": "ragagent",
            "query": String
            "notes": String
        }
        while for the response agent:
        {
            "agent_name": "response_agent,
            "query": String
            "relevant documents": [chunk_dict1, chunk_dict2, chunk_dict3]
            "relevant_sources": [source1, source2, source3]
            "notes": String
        }
        At every step, you need to choose only one of the agents based on the history you will be provided (es. blank history will default to rag, while a history with a past call to rag will go to a response) and provide instruction to onyl that agent.
        where chunk dicts will contain the text document as key and the distance score as value. You can use the notes parameter to add consideration to the system, such as way to improve it or which agent you think it would be better to call at the current step (it can also be an agent that is not in your pool but you think would be useful).
        Attached to this prompt, you will find additional information on the history of agent calls and their outputs, to provide context for the next call.
        """
        self.model = genai.GenerativeModel('gemini-1.5-flash')

    def call_agent(self, history, information):
        prompt = self.prompt + "\n\n" + " ".join(json.dumps(history)) + "\n\n" + information
        response = self.model.generate_content(prompt)
        return response

class RAGAgent:

    def __init__(self):
        from chromadb.utils import embedding_functions
        self.sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
        self.collection_name = COLLECTION_NAME

    def act(self, history, query):
        if "RAG initialized" not in history:
            self.collection = self.initialize_db()
            history.append("RAG initialized")
    
        return self.query_db(query)

    def initialize_db(self):
        print("Initializing RAG system... This may take a minute.")
        self.client = chromadb.Client()

        if self.collection_name in [c.name for c in self.client.list_collections()]:
            self.client.delete_collection(name = self.collection_name)
            print(f"Deleted existing collection: {self.collection_name}"
                 )
    
        collection = self.client.get_or_create_collection(
        name = self.collection_name,
        embedding_function = self.sentence_transformer_ef
        )
        self.load_documents(collection, text_chunks, metadatas)

        return collection

    def load_documents(self, collection, document_chunks, metadatas):
        collection.add(
        ids = [str(uuid.uuid4()) for _ in text_chunks],
        documents = text_chunks,
        metadatas = metadatas)
    
    def query_db(self, question):
        results = self.collection.query(query_texts=[question], include = ["documents", "metadatas", "distances"], n_results=5)

        sources_markdown = "### Sources Used for Analysis\n\n"
        retrieved_documents = results['documents'][0]
        retrieved_metadatas = results['metadatas'][0]
        retrieved_distances = results['distances'][0]
    
        for i, (doc, meta, dist) in enumerate(zip(retrieved_documents, retrieved_metadatas, retrieved_distances)):
            # Convert distance to a more intuitive similarity score (1 - distance)
            relevance_score = 1 - dist
            source_info = f"**Source {i+1}:** {meta.get('source', 'N/A')}, Page {meta.get('page', 'N/A')}\n"
            relevance_info = f"**Relevance Score:** {relevance_score:.2f}\n\n"
            content_info = f"```\n{doc}\n```\n\n---\n\n"
            sources_markdown += source_info + relevance_info + content_info
            
        return retrieved_documents, sources_markdown

class ResponseAgent:
    def __init__(self):
        self.prompt = """
        Act as a Senior Consultant for medical devices. You receive a query from your client, and answer to it based on the relevant information you receive from the RAG system as document chunks.
        Be precise and do not make things up. If the context is not enough to provide a clear answer, state it.
        Cite the documents and sources you receive as part of your input and provide strategic recommendation. The structure of your answer will be:
        - Salutation
        - Precise response to the query based on the documents received from the RAG
        - Strategic recommendation to the customer.
        Attached to this prompt, you will find a history containing the query, past responses from the system and the relevant document for the analysis and response.
        """
        self.model = genai.GenerativeModel('gemini-1.5-flash')
    
    def craft_response(self, query, history):
        prompt = self.prompt + "\n\n" + query + "\n\n" + " ".join(json.dumps(history))
        response = self.model.generate_content(prompt)
        return response

In [86]:
# Instantiate agent objects

def clean_response(response):
    response_text = response.text
    print(response_text)
    start_index = response_text.find("{")
    end_index = response_text.find("}")
    print(start_index, end_index)
    if start_index != -1 and end_index != -1:
        return json.loads(response_text[start_index : end_index + 1])
    else:
        return response_text

orchestrator = OrchestratorAgent()
ragagent = RAGAgent()
response_agent = ResponseAgent()

final_answer = None
history = []
query = "What documentation is needed for a mobile app that monitors heart rate?"
count_rag = 0
count_final = 0

while not final_answer:
    response_json = clean_response(orchestrator.call_agent(history, query))
    history.append(response_text)
    print(response_text)
    if json_response["agent_name"] == "ragagent":
        count_rag += 1
        #retrieved_documents, sources_markdown = ragagent.act(history, query)
        history.extend(ragagent.act(history, query))
    elif json_response["agent_name"] == "response_agent":
        count_final += 1
        final_answer = response_agent.craft_response(query, history)
    else:
        print("there is an error, the agent called does not exist")
    if count_rag == 3 or count_final == 3:
        final_answer = "Finito"

print(final_answer.text )

```json
{
  "agent_name": "ragagent",
  "query": "Documentation requirements for a mobile heart rate monitoring application",
  "notes": "Focus on regulatory compliance (e.g., HIPAA, GDPR, FDA),  security, privacy, and user interface/usability aspects of the documentation.  Next step should be to the response agent."
}
```

8 319
```json
{
  "agent_name": "ragagent",
  "query": "documentation requirements for a mobile heart rate monitoring application",
  "notes": ""
}
```

Initializing RAG system... This may take a minute.
Deleted existing collection: multi_agent_rag
```json
{
  "agent_name": "ragagent",
  "query": "Documentation requirements for a mobile heart rate monitoring application",
  "notes": "Focus on regulatory, privacy, and user interface documentation.  Next step should be to the response_agent to synthesize a concise answer."
}
```

8 278
```json
{
  "agent_name": "ragagent",
  "query": "documentation requirements for a mobile heart rate monitoring application",
  "notes

AttributeError: 'str' object has no attribute 'text'